<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Предварительное-изучение-данных" data-toc-modified-id="Предварительное-изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Предварительное изучение данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Для защиты сведений о клиентах страховой компании необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Основное требование - чтобы при преобразовании качество моделей машинного обучения не ухудшилось, поэтому нужно обосновать корректность работы решения. Подбирать наилучшую модель в рамках проекта не требуется.

**Цель проекта**

Разработать алгоритм преобразования данных для защиты персональной информации клиентов без ухудшения прогнозных свойств.

**Задачи**

1. Изучить предоставленные наборы данных, общие характеристики клиентов. Выявить возможные пропуски, аномальные значения, дубликаты и другие особенности, препятствующие адекватной реализации проекта.
2. Изучить влияние на качество прогнозирования умножения признаков на обратимую матрицу. Обосновать, почему качество множественной регрессии не поменяется / изменится.
3. Предложить алгоритм преобразования данных для защиты информации. Обосновать решение.
4. Запрограммировать алгоритм, применив матричные операции. Проверить, что качество линейной регрессии не отличается до и после преобразования, применив метрику R2.

## Предварительное изучение данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from scipy import stats as st
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer

import warnings
warnings.filterwarnings("ignore")

Загрузим данные и посмотрим на начало таблицы.

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')
display(insurance.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Выведем основную информацию по датафрейму.

In [3]:
print(insurance.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


В наборе данных 5 колонок (один из столбцов - целевой признак Страховые выплаты). Пропущенных значений нет, типы данных в целом корректы, кроме возраста. Возможно, имеет смысл перевести столбец с возрастом в целочисленный формат (int).

Рассмотрим общую статистику по клиентам.

In [4]:
display(insurance.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Явных артефактов и экстремальных значений в данных нет. В столбце пол минимальное значение 0, максимальное 1. Соотношение клиентов по полу примерно одинаково (среднее значение 0.499, в середине между нулем и единицей). Минимальный возраст - 18 лет, максимальный - 65, средний - около 31 года. Здесь тоже нет сюрпризов. Среди застрахованных есть как одиночки (0), так и клиенты с 6 членами семьи. 

Что касается целевого признака (количество страховых выплат клиенту за последние 5 лет), то как минимум 75% клиентов не обращаются за страховыми выплатами. Однако, в наборе данных есть индивидуумы которые умудрились обратиться 5 раз за пять лет.

Подготовим данные к работе. Изменим названия заголовков на англоязычные, все со строчной буквы.

In [5]:
insurance.columns = ['gender', 'age', 'income', 'family', 'insurance_payments']

Переведем столбец age в целочисленный формат.

In [6]:
insurance['age'] = insurance['age'].astype(int)

Проверим, сколько в наборе данных имеется дубликатов.

In [7]:
print('Количество дубликатов в наборе данных: {}'.format(insurance.duplicated().sum()))

Количество дубликатов в наборе данных: 153


In [8]:
display(insurance[insurance.duplicated()].sort_values(by='age', ascending=False))

      gender  age   income  family  insurance_payments
2869       0   50  24700.0       1                   2
3923       1   41  48900.0       0                   0
2274       1   41  30200.0       0                   0
1485       1   41  32700.0       0                   0
3365       1   41  47100.0       1                   0
...      ...  ...      ...     ...                 ...
3419       1   19  41600.0       1                   0
4935       1   19  32700.0       0                   0
4129       1   19  35600.0       2                   0
2269       1   19  43200.0       1                   0
2429       1   18  39800.0       2                   0

[153 rows x 5 columns]


Не совсем ясна природа этих повторов. Они могли появиться в датасете случайно из-за того, что выгружались из разных баз или менеджер при оформлении страховки ошибочно продублировал карточку клиента. С другой стороны, клиенты могли приобретать услуги страховой компании несколько раз.

В проекте нет задачи улучшить качество модели, нужно только преобразовать данные таким образом, чтобы это не ухудшило качество прогнозирования. Наличие повторов не повлияет на сам алгоритм преобразования, и удалять их не имеет смысла. Оставим дубликаты в датасете, просто зафиксируем факт их наличия.

Разобьем набор данных на обучающую и тестовую выборки. Тестовая выборка будет составлять 25% от всего датасета.

In [9]:
insurance_train, insurance_test = train_test_split(insurance, test_size=0.25, random_state=12345)

Полученные выборки, в свою очередь, разделим на набор влияющих факторов и целевой признак.

In [10]:
target_train = pd.DataFrame(insurance_train, columns = ['insurance_payments'])
features_train = insurance_train.drop(['insurance_payments'], axis=1)
target_test = pd.DataFrame(insurance_test, columns = ['insurance_payments'])
features_test = insurance_test.drop(['insurance_payments'], axis=1)

print('train', features_train.shape, target_train.shape)
print('test', features_test.shape, target_test.shape)

train (3750, 4) (3750, 1)
test (1250, 4) (1250, 1)


### Вывод

В предоставленном наборе данных 5000 строк и 5 колонок.
Пропусков нет, как нет и явных выбросов или артефактов.

Для удобства работы с данными заголовки были заменены на англоязычные.
Тип данных в колонке с возрастом изменен на целочисленный.

В исходной таблице обнаружено 153 дубликата. Их происхождение не ясно, это могло быть задвоение при выгрузке из разных баз или же клиент оформлял страховку в компании не единожды и был посчитан несколько раз. Поскольку наличие дубликатов никак не отражается на преобразовании записей, повторы оставили в датасете. 

## Умножение матриц

Посмотрим, как умножение признаков на обратимую матрицу повлияет на качество линейной регрессии, изменятся ли результаты прогнозирования или нет.

Ниже приведены значения используемых символов.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $a$  — прогнозируемый вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование**

В задаче требуется ответить, как повлияет умножение матрицы признаков $X$ на случайную обратимую матрицу $P$. Т.е. необходимо вывести, что $a_x = a_p$, либо $a_x != a_p$. Для ответа на вопрос также воспользуемся формулой расчета предсказаний $a = Xw$ и свойствами транспонирования, обратной матрицы и ассоциативностью.

В выражение для расчета вектора весов линейной регрессии $w = (X^T X)^{-1} X^T y$ вместо $Х$ подставим $ХР$. В начале нашего решения приравняем $a_x$ к $a_p$.

$$
a_x = a_p
$$

$$
a_x = Xw_x,    a_p = XPw_p 
$$

$$
Xw_x = XPw_p
$$

$$
w_x = Pw_p
$$

$$
w_p = ((XP)^T (XP))^{-1}  (XP)^T y = (P^T  X^T  X P)^{-1} (XP)^T y = (P^T  (X^T  X) P)^{-1} (XP)^T y = 
P^{-1}  (X^T  X)^{-1} (P^T)^{-1}  P^T X^T y = P^{-1}  (X^T  X)^{-1}  X^T y = P^{-1}  X^{-1} (X^T)^{-1} X^T y = P^{-1}  X^{-1} y = (P)^{-1}  (X)^{-1}  Xw_x = (P)^{-1}  w_x
$$

Далее в выражение $a_p = XPw_p$ подставляем получившееся значение вектора $w_p$.

$$
a_p = XP (P)^{-1}  w_x = Xw_x = a_x
$$

$$
a_p = a_x
$$

**Ответ** 

Как показал вывод из преобразований, умножение матрицы признаков на случайную обратимую матрицу не повлияет на конечный результат прогнозирования.

## Алгоритм преобразования

Наш целевой признак - это количество страховых выплат клиенту за последние 5 лет. Т.е. это задача регрессии, а не классификации (в этом случае требовалось бы определить, обращался ли клиент за страховой выплатой или нет). Для прогнозирования выберем модель линейной регрессии, т.к. ее проще интерпретировать, и выведем коэффициент детерминации.

Сначала посчитаем целевую метрику R2 для модели, обученной по исходным (неизмененным) данным. Сделаем признаки матрицей с 5 столбцами, добавив в нее столбец единиц. Его можно рассматривать как постоянный признак, который при умножении на соответствующий коэффициент становится параметром сдвига или пересечением подобранной линии с осью $y$.

In [11]:
class LinearRegressionTest:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegressionTest()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('r2_score:', r2_score(target_test, predictions).round(4))

r2_score: 0.4352


**Алгоритм**

Теперь посмотрим, что будет с результатами, если преобразовать наши данные. Создадим случайную матрицу для умножения на матрицу признаков. Сохраним ее в переменной P.

In [12]:
P = np.random.randint(10, size=(features_train.shape[1], features_train.shape[1]))
print(P)
print(P.shape)

[[4 0 4 3]
 [1 7 0 2]
 [5 7 3 5]
 [3 4 1 9]]
(4, 4)


Проверим получившуюся матрицу на вырожденность. Для этого вычислим определитель получившийся мартицы.

In [13]:
print(np.linalg.det(P))

-168.99999999999986


Определитель не равен нулю, значит, матрица является обратимой, т.е. невырожденной. 

Посмотрим, как будет выглядеть персональная информация клиентов (train_features и test_features), умноженная на получившуюся обратимую матрицу.

In [14]:
display(features_train.dot(P), features_test.dot(P))

             0         1         2         3
3369  181050.0  253705.0  108605.0  181098.0
1441  288038.0  403438.0  172804.0  288071.0
571   205535.0  287928.0  123301.0  205573.0
225   225539.0  315956.0  135301.0  225581.0
2558  253039.0  354439.0  151802.0  253084.0
...        ...       ...       ...       ...
3497  160542.0  224994.0   96300.0  160584.0
3492  113540.0  159112.0   68104.0  113592.0
2177  223548.0  313191.0  134105.0  223594.0
3557  250534.0  350870.0  150304.0  250580.0
4578  204019.0  285733.0  122400.0  204038.0

[3750 rows x 4 columns]              0         1         2         3
3183  195045.0  273247.0  117004.0  195102.0
1071  215556.0  302058.0  129302.0  215618.0
2640  210543.0  294973.0  126304.0  210581.0
2282  174020.0  243740.0  104400.0  174040.0
1595  200053.0  280303.0  120004.0  200118.0
...        ...       ...       ...       ...
982   145061.0  203365.0   87006.0  145123.0
3820  289546.0  405543.0  173707.0  289596.0
3595  211539.0  296345.0  1269

Получился набор самых разных значений, из которых, на первый взгляд, сложно выделить какой-то смысл.

Посчитаем, каков будет R2 для модели, обученной на нашей матрице обучающих признаков (train_features) помноженной на получившуюся обратимую матрицу.

In [15]:
class LinearRegressionTest:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones(((train_features).dot(P).shape[0], 1)), train_features.dot(P)), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(P).dot(self.w) + self.w0
    
model = LinearRegressionTest()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('r2_score:', r2_score(target_test, predictions).round(4))

r2_score: 0.4352


**Обоснование**

R2_score не изменился. Это значит, что мы можем применять этот метод для защиты личной информации клиентов без ущерба для прогнозирования. Подобный алгоритм выполняет основную функцию (защищает персональные данные), к тому же прост и удобен в использовании, позволяет оперировать преобразованными данными для предсказания результатов (в данном случае, количество страховых выплат) и не портит качество прогнозов.

Для большей убедительности проверим работу алгоритма на тестовой выборке с помощью модели линейной регрессии, загруженной из scikit-learn.

### Вывод

В разделе предложен алгоритм решения задачи преобразования данных. Матрица признаков умножается на случайную матрицу, после чего к ней добавляется столбец единиц. На полученном наборе и обучается модель, для предсказания которой подается уже новый набор данных, умноженный на такую же обратимую матрицу, как и обучающий датасет. Использование обратимых матриц для преобразования исходных данных не влияет на качество прогнозов.

## Проверка алгоритма

Для проверки алгоритма сформируем обучающий датасет из преобразованной информации: умножим наши обучаюшие и тестовые выборки на обратимую матрицу.
Для обучения и проверки базовой модели (на исходных, непреобразованных данных) признаки у нас уже есть (features_train и target_train)

In [16]:
features_train_exp = features_train.dot(P)
features_test_exp = features_test.dot(P)

Обучим модели по датасетам. Выведем часть предиктов, не изменились ли они.

In [17]:
model_base = LinearRegression()
model_base.fit(features_train, target_train)
print(model_base.predict(features_test))

model_exp = LinearRegression()
model_exp.fit(features_train_exp, target_train)
print(model_exp.predict(features_test_exp))

[[0.17494798]
 [0.80523476]
 [0.45599281]
 ...
 [0.3129923 ]
 [0.34926113]
 [0.7886826 ]]
[[0.17494798]
 [0.80523476]
 [0.45599281]
 ...
 [0.3129923 ]
 [0.34926113]
 [0.7886826 ]]


Прогнозы остались прежними даже при измененных исходных данных. Объявим функцию для подсчета R2 и других основных метрик для моделей.

In [18]:
def compare_table(models, features, target):
    df = pd.DataFrame(columns=['models', 'MSE', 'RMSE', 'R2'])
    for name, model in models.items():
        prediction = model.predict(features[name])
        mse = mean_squared_error(target, prediction).round(3)
        rmse = (mse ** 0.5).round(3)
        r2 = r2_score(target, prediction).round(3)
        df = df.append({'models': name, 'MSE': mse, 'RMSE': rmse, 'R2': r2}, ignore_index=True)
    return df

Создадим списки моделей и тестовых выборок для предсказания целевого признака и применим функцию для расчета метрик.

In [19]:
models_list = {'LR base': model_base, 'LR experimental': model_exp}
features_list = {'LR base': features_test, 'LR experimental': features_test_exp}

In [20]:
print(compare_table(models_list, features_list, target_test))

            models    MSE   RMSE     R2
0          LR base  0.117  0.342  0.435
1  LR experimental  0.117  0.342  0.435


### Вывод

Для проверки работы метода преобразования информации была сравнена эффективность модели по двум наборам данных: контрольному (где данные не подвергались изменениям) и измененному (где факторы были умножены на обратимую матрицу). В результате эксперимента целевая метрика R2 не поменяллась. Алгоритм можно применять для защиты персональной информации клиентов компании.

### Общий вывод

В работе рассмотрен метод преобразования персональной информации. Для ее защиты предлагается применить случайную обратимую матрицу. Матрица признаков умножается на эту обратимую матрицу, и на получившихся данных обучается модель. Далее при прогнозировании результатов по новым данным необходимо провести точно такие же преобразования и с новыми данными. Целевой признак при обучении модели домножать не нужно.

С помощью вывода формул, а также экспериментальным путем установлено, что качество прогнозов множественной линейной регрессии не отличается до и после преобразования. Результаты оценивались при помощи метрики R2.